In [ ]:
import pandas as pd
import csv
import numpy as np
import random
from pprint import pprint
from typing import Callable
from functools import total_ordering
from heapq import heappush
from collections import deque
from scipy.stats import t
df = pd.read_csv('start_station_probs.csv')

d = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
stations = []
prob = []
for s, p in d.items():
  stations.append(s)
  prob.append(p)

csv_file_path = "trip_stats.csv"

probs = {}

with open(csv_file_path) as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)
    data = {}
    for row in csv_reader:
        start = row[0]
        end = row[1]
        count = int(row[2])
        if start not in data:
            data[start] = {"end_points": [], "probs": []}
        data[start]["end_points"].append(end)
        data[start]["probs"].append(count)

    for start, d in data.items():
        total_count = sum(d["probs"])
        probs[start] = [d["end_points"], [p / total_count for p in d["probs"]]]

print(probs['South Waterfront Walkway - Sinatra Dr & 1 St'])
R = 3500

def randomGenerator():
  result = []
  for _ in range(R):
     i = random.choices(stations, weights=prob)[0]
     j = random.choices(probs[i][0], weights=probs[i][1])[0]
     result.append([i,j])
  return result


def staticVars(**args):
    def decorate(func):
        for k in args:
            setattr(func, k, args[k])
        return func
    return decorate

@staticVars(t=0)
def now():
    return now.t

print("The current time is", now())

def setTime(t_new=0):
    now.t = t_new
    return now()

@total_ordering
class Event:
    def __init__(self, t: int, i: int, j: int, f: Callable = None):
        self.t = t
        self.i = i
        self.j = j
        self.f = f

    def __lt__(self, other):
        if not isinstance(other, Event):
            return NotImplemented
        if self.t < other.t:
            return True
        elif self.t == other.t:
            if self.f.__name__ is not None and other.f.__name__ is not None:
                return self.f.__name__< other.f.__name__
                return True
        return False

    def __eq__(self, other):
        if not isinstance(other, Event):
            return NotImplemented
        return self.t == other.t and self.i == other.i and self.j == other.j



class FutureLi:
    def __init__(self):
        self.events = []

    def __iter__(self):
        return self

    def __next__(self) -> Event:
        from heapq import heappop
        if self.events:
            return heappop(self.events)
        raise StopIteration

    def __repr__(self) -> str:
        from pprint import pformat
        return pformat(self.events)

def schedule(e: Event, fev: FutureLi):
    heappush(fev.events, e)


def simulate(state, event_list, verbose=True):
    arrived_count = 0
    riding_count = 0
    alist = []

    for e in event_list:
        if e.t > 1440:
            break
        setTime(e.t)
        e.f(state,e.i,e.j, event_list)
        if e.f.__name__=="arrived_startpoint":
            arrived_count+=1
        if e.f.__name__=="start_riding":
            riding_count+=1


    avg_waiting = state['waitTime'] / riding_count
    alist.append(riding_count*100/arrived_count)
    alist.append(avg_waiting)
    return alist

def initial_state(num = 10):

    num_bikes = {}
    infoWaiter = {}
    for i in stations:
      num_bikes[i] =  num
      infoWaiter[i] = deque([])
    return {'num_bikes': num_bikes
            ,'infoWaiter': infoWaiter
            ,'waitTime' : 0}



def arrived_startpoint(s,i,j, fev):
    s['infoWaiter'][i].append([j, now()])
    if s['num_bikes'][i] > 0:
        schedule(Event(now(),i,j, start_riding), fev)


def start_riding(s,i,j, fev):
    assert s['num_bikes'][i] > 0
    assert len(s['infoWaiter'][i]) > 0
    s['num_bikes'][i] -= 1
    arrived_time = s['infoWaiter'][i].popleft()[1]
    s['waitTime'] += now() - arrived_time

    mu = 2.78
    sigma = 0.619
    total = 0

    ride_times = np.random.lognormal(mu, sigma, 10)
    for x in ride_times:
      total += x
    total /= 10
    schedule(Event(now() + total,i,j, arrived_endpoint), fev)
    if len(s['infoWaiter'][i]) > 0 and s['num_bikes'][i] > 0:
        schedule(Event(now(),i,j, start_riding), fev)



def arrived_endpoint(s,i,j, fev):
    s['num_bikes'][j] = s['num_bikes'].get(j,0) + 1
    if len(s['infoWaiter'].get(j,[])) > 0 and s['num_bikes'].get(j,0) > 0:
         schedule(Event(now(),j, s['infoWaiter'][j][0][0], start_riding), fev)



randomij = randomGenerator()
timeList = [t for t in range(R)]


lambda_rate = 2.38
interarrival_times = np.random.exponential(scale=1/lambda_rate, size=R)
arrival_times = np.cumsum(interarrival_times)

state = initial_state()
event_list = FutureLi()
for k in range(R):
    i, j = randomij[k]
    t = arrival_times[k]
    schedule(Event(t, i, j, arrived_startpoint), event_list)

simulate(state, event_list)


def final_simulator():
  success_rate_list = []
  wait_time_list = []
  MaxWaiter = {}
  for x in range(30):
    randomij = randomGenerator()
    timeList = [t for t in range(R)]
    import numpy as np

    lambda_rate = 2.38
    interarrival_times = np.random.exponential(scale=1/lambda_rate, size=R)
    arrival_times = np.cumsum(interarrival_times)
    state = initial_state(10)
    event_list = FutureLi()
    for k in range(R):
      i, j = randomij[k]
      t = arrival_times[k]
      schedule(Event(t, i, j, arrived_startpoint), event_list)
    res = simulate(state, event_list)
    success_rate_list.append(res[0])
    wait_time_list.append(res[1])


  inputArr = success_rate_list
  sample_mean = np.mean(inputArr)
  sample_std = np.std(inputArr, ddof=1)

  df = len(inputArr) - 1
  alpha = 0.1
  t_crit = t.ppf(1 - alpha / 2, df)

  margin_of_error = t_crit * sample_std / np.sqrt(len(inputArr))

  lower_bound = sample_mean - margin_of_error
  upper_bound = sample_mean + margin_of_error

  print(f"The 90% confidence interval is ({lower_bound:.2f}, {upper_bound:.2f})")
  inputArr = wait_time_list
  sample_mean = np.mean(inputArr)
  sample_std = np.std(inputArr, ddof=1)

  df = len(inputArr) - 1
  alpha = 0.1
  t_crit = t.ppf(1 - alpha / 2, df)

  margin_of_error = t_crit * sample_std / np.sqrt(len(inputArr))

  lower_bound = sample_mean - margin_of_error
  upper_bound = sample_mean + margin_of_error

  print(f"The 90% confidence interval is ({lower_bound:.2f}, {upper_bound:.2f})")

final_simulator()


def idealized_simulator():
  state = initial_state(1)
  start_bikes = dict(state['num_bikes'])
  c = 0
  while c < 10:
    randomij = randomGenerator()
    timeList = [t for t in range(R)]
    import numpy as np
    lambda_rate = 2.38
    interarrival_times = np.random.exponential(scale=1/lambda_rate, size=R)
    arrival_times = np.cumsum(interarrival_times)
    event_list = FutureLi()
    for k in range(R):
      i, j = randomij[k]
      t = arrival_times[k]
      schedule(Event(t, i, j, arrived_startpoint), event_list)
    state['num_bikes'] = dict(start_bikes)
    res = simulate(state, event_list)
    for i, waiter in state['infoWaiter'].items():
      if len(waiter) > 0:
        start_bikes[i] += 1
        c=0
    state = initial_state()
    c+=1
    print(start_bikes)
    print(res)
  print(len(start_bikes))
  return start_bikes

idealized_simulator()

[['11 St & Washington St', '12 St & Sinatra Dr N', '14 St Ferry - 14 St & Shipyard Ln', '4 St & Grand St', '5 Corners Library', '6 St & Grand St', '7 St & Monroe St', '8 St & Washington St', '9 St HBLR - Jackson St & 8 St', 'Adams St & 11 St', 'Adams St & 2 St', 'Baldwin at Montgomery', 'Bergen Ave', 'Bergen Ave & Stegman St', 'Bloomfield St & 15 St', 'Brunswick & 6th', 'Brunswick St', 'Christ Hospital', 'Church Sq Park - 5 St & Park Ave', 'City Hall', 'City Hall - Washington St & 1 St', 'Clinton St & 7 St', 'Clinton St & Newark St', 'Columbus Dr at Exchange Pl', 'Columbus Drive', 'Columbus Park - Clinton St & 9 St', 'Dey St', 'Dixon Mills', 'Essex Light Rail', 'Glenwood Ave', 'Grand St', 'Grand St & 14 St', 'Grove St PATH', 'Hamilton Park', 'Harborside', 'Heights Elevator', 'Hilltop', 'Hoboken Ave at Monmouth St', 'Hoboken Terminal - Hudson St & Hudson Pl', 'Hoboken Terminal - River St & Hudson Pl', 'Hudson St & 4 St', 'JC Medical Center', 'Jersey & 3rd', 'Jersey & 6th St', 'Lafayette

{'South Waterfront Walkway - Sinatra Dr & 1 St': 35,
 'Grove St PATH': 33,
 'Hoboken Terminal - Hudson St & Hudson Pl': 39,
 'Hoboken Terminal - River St & Hudson Pl': 37,
 'Newport Pkwy': 33,
 'City Hall - Washington St & 1 St': 33,
 'Newport PATH': 32,
 '12 St & Sinatra Dr N': 28,
 'Hoboken Ave at Monmouth St': 31,
 'Marin Light Rail': 27,
 'Hamilton Park': 34,
 '14 St Ferry - 14 St & Shipyard Ln': 27,
 'Liberty Light Rail': 27,
 'Columbus Dr at Exchange Pl': 26,
 'Harborside': 28,
 '11 St & Washington St': 26,
 'Washington St': 27,
 'Sip Ave': 27,
 'Hudson St & 4 St': 25,
 '8 St & Washington St': 26,
 'Madison St & 1 St': 25,
 'City Hall': 27,
 'Warren St': 26,
 'Newark Ave': 25,
 'Columbus Park - Clinton St & 9 St': 26,
 'Grand St & 14 St': 24,
 'Church Sq Park - 5 St & Park Ave': 25,
 'Columbus Drive': 23,
 'Van Vorst Park': 21,
 'Clinton St & Newark St': 21,
 'Grand St': 21,
 'Paulus Hook': 21,
 'Manila & 1st': 20,
 '9 St HBLR - Jackson St & 8 St': 24,
 'Bloomfield St & 15 St': 2